Self check template. (this cell can and should be removed in selfcheck ntoebooks )


In [ ]:
# edit the strings to describe the check
result = {
    "area": "spark",
    "description": "Check that spark can use delta tables.",
    "passed": False,
    "message": "",
    "plugin": "spark",
}

In [ ]:
from freeds.spark import get_spark_session, show_spark_info

try:
    spark = get_spark_session("self-check1")
    data = spark.range(100)
    selfcheck_db = "selfcheckdb"
    table_name = f"{selfcheck_db}.wikipedia_page_reads"

    # create some data in delta
    spark.sql(f"DROP DATABASE IF EXISTS {selfcheck_db} CASCADE")
    spark.sql(f"CREATE DATABASE IF NOT EXISTS {selfcheck_db}")

    (
        data.write.option(  # .mode("overwrite")  # Options: 'overwrite', 'append', 'ignore', 'error' (default)
            "mergeSchema", "true"
        )
        .format("delta")  # Options: 'parquet', 'csv', 'json', 'orc', etc.
        .saveAsTable(table_name)
    )
    spark.stop()

    # read some data in delta
    spark = get_spark_session("self-check2")
    data = spark.table(table_name)
    show_spark_info(spark)
    data.show(5)

    # clean up
    spark.sql(f"DROP DATABASE IF EXISTS {selfcheck_db} CASCADE")
    spark.stop()

    result["message"] = "Executed spark cell ok."
    result["passed"] = True

except Exception as ex:
    result["message"] = str(ex)
    result["passed"] = False

In [ ]:
import json

print(json.dumps(result, indent=4))